In [27]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import threading
import queue

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CSFic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CSFic\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
data_set = {"intents": [{"tag": "greeting",
         "patterns": ["Jarvis are you there", "Hi", "Help", "Help Please" "Hi there", "How are you", "Is anyone there","Hey","Hola", "Hello", "Good day", "You up", "Come in", "can you hear me"],
         "responses": ["Greetings Mr. Starke, how can I be of assistance?", "At your service sir.", "I am online sir, how can I help?"],
         "context": [""]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time", "Systems off"],
         "responses": ["Have a nice day sir", "Going offline.", "It has been a pleasure."],
         "context": [""]
        },
        {"tag": "thanks",
         "patterns": ["Wonderful", "That is perfect", "Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
         "responses": ["You are most welcome.", "Of course, it is my pleasure"],
         "context": [""]
        },
        {"tag": "noanswer",
         "patterns": [" "],
         "responses": ["Communications failure.", "Commmunications are offline.", "You are not authorized to access this area."],
         "context": [""]
        },
        {"tag": "broad questions",
         "patterns": ["Are we ready"],
         "responses": ["All systems are a go."],
         "context": [""]
        },
        {"tag": "Enemies",
         "patterns": ["Update on Enemies", "Where are the bad guys"],
         "responses": ["Thanos is nearing Earth's orbit now.", "There have been some reports of alien life in New York City.", "The Mandarin's location is currently unknown."],
         "context": [""]
        },
        {"tag": "Allies",
         "patterns": ["Who are the Avengers", "Who are my friends", "Allies", "Who are my allies"],
         "responses": ["Labeled \"Earth's Mightiest Heroes\", the Avengers consisted of Iron Man, Ant-Man, Hulk, Thor and the Wasp. The original Captain America was discovered trapped in ice in issue #4, and joined the group after they revived him.", "Some of your contacts include Hawkeye, Black Panther, Namor, Scarlett Witch, Spider-man, Falcon, Hulk, Quiksilver, Doctor Strange, and Nick Fury."],
         "context": [""]
        },
        {"tag": "Systems Check",
         "patterns": [ "How are the systems", "Status report", "systems report", "Weapons Status", "How are we looking"],
         "responses": ["Power is at maximum capacity sir.", "Systems are at critical levels sir.", "Omnibeam is fully functional.", "Weapons are fully operational.", "Importing preferences and calibrating virtual environment."],
         "context": [""]
        },
        {"tag": "topics",
         "patterns": ["topics"],
         "responses": ["Currently programmed to respond to Greetings, Systems check, questions, trivia, thanks, no answer, enemies, allies, movie quotes, and goodbyes. For more elaboration type the topic for more examples."],
         "context": [""]
        },
        {"tag": "Movie Quotes",
         "patterns": ["What are some Avenger Quotes", "Quotes"],
         "responses": ["The hardest choices require the strongest wills. ~ Thanos", "Part of the journey is the end. ~ Tony Stark", "No amount of money ever bought a second of time. ~ Tony Stark", "You know, I keep telling everybody they should move on and grow. Some do. But not us. ~ Steve Rogers", "It's not about how much we lost. It's about how much we have left.~ Tony Stark"],
         "context": [""]
        },
        {"tag": "Marvel Trivia",
         "patterns": ["Trivia", "Tell me about the Avengers", "Tell me something cool", "Tell me something interesting", "Tell me Trivia.", "Tell me something"],
         "responses": ["JARVIS was based on the character with the same name in the comics, which was Edwin Jarvis.", "JARVIS is one of the five known A.I. characters to have a main role so far in the whole film franchise.", "JARVIS was portrayed and voiced by actor Paul Bettany throughout the course of the franchise, including The Avengers.", "The Wasp gave the Avengers team their name and was a founding member.","In the comics Hulk was only a member for one issue.","The Vision was originally supposed to be an alien."],
         "context": [""]
        },
                        # add examples of questions you can ask
        {"tag": "Questions Example",
         "patterns": ["What questions can I ask", "Questions" ],
         "responses": ["Are we ready? What can you do? Where am I? Who are you? What support is offered? How much power do we have left? How are the systems? Status report? Weapons Status?"],
         "context": [""]
        },
                        # specify marvel characters specificly 
        {"tag": "Hulk",
         "patterns": ["Who is the Hulk", "Hulk", "Bruce Banner", "Who is Bruce Banner"],
         "responses": ["Dr. Bruce Banner, Ph.D. a.k.a. the Hulk is an American theoretical physicist, famed for his work into the studies of nuclear physics and gamma radiation.", "He is a mutagenically altered human of immense strength, toughness, durability, speed and healing factor."],
         "context": [""]
        },
                        # specific Questions
        {"tag": "Specific 1",
         "patterns": ["Who am I"],
         "responses": ["You are Tony Stark, Iron Man."],
         "context": [""]
        },
                        
        {"tag": "Specific 2",
         "patterns": ["What can you do", "What are your functions"],
         "responses": ["I can do anything you program me to do.", "My functions are to assist your needs, including systems check and retreiving information."],
         "context": [""]
        },
                        
        {"tag": "Specific 3",
         "patterns": ["Who are you", "Who is speaking", "What is your name", "Who is this"],
         "responses": ["I am JARVIS, I am an AI that functions as Tony Stark's assistant, running and taking care of all the internal systems of Stark's buildings and the Iron Man suits"],
         "context": [""]
        },
                        
        {"tag": "Specific 4",
         "patterns": ["Where am I"],
         "responses": ["You are currently located at the Avengers compound.", "Current location is Starke Industries."],
         "context": [""]
        },
                        
        {"tag": "Specific 5",
         "patterns": ["Where is Happy Hogan", "Happy's location", "I miss Happy", "Happy"],
         "responses": ["Happy is currently meeting with Aunt May.", "Happy is with Spider-man."],
         "context": [""]
        },
                        
        {"tag": "Specific 6",
         "patterns": ["Where is Pepper"],
         "responses": ["Pepper is currently meeting with Justin Hammer at Hammer Industries.", "Pepper is currently with Obadiah Stane at an Awards ceremony."],
         "context": [""]
        },
                        
        {"tag": "Specific 7",
         "patterns": ["Engage Heads Up Display"],
         "responses": ["Check."],
         "context": [""]
        },                        
                        
        {"tag": "Specific 8",
         "patterns": [""],
         "responses": [""],
         "context": [""]
        },
                        
        {"tag": "Specific 9",
         "patterns": [""],
         "responses": [""],
         "context": [""]
        },
                        
        {"tag": "Specific 10",
         "patterns": [""],
         "responses": [""],
         "context": [""]
        },
                        
        {"tag": "Specific 11",
         "patterns": [""],
         "responses": [""],
         "context": [""]
        }
            
                
   ]
}

data_file = json.dumps(data_set)

with open("intents.json", "w") as outfile:
    json.dump(data_set, outfile)

In [29]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [30]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [31]:
import nltk
nltk.download('omw-1.4')

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

72 documents
24 classes ['Allies', 'Enemies', 'Hulk', 'Marvel Trivia', 'Movie Quotes', 'Questions Example', 'Specific 1', 'Specific 10', 'Specific 11', 'Specific 2', 'Specific 3', 'Specific 4', 'Specific 5', 'Specific 6', 'Specific 7', 'Specific 8', 'Specific 9', 'Systems Check', 'broad questions', 'goodbye', 'greeting', 'noanswer', 'thanks', 'topics']
94 unique lemmatized words ["'s", ',', '.', 'about', 'ally', 'am', 'anyone', 'are', 'ask', 'avenger', 'awesome', 'bad', 'banner', 'bruce', 'bye', 'can', 'chatting', 'come', 'cool', 'day', 'display', 'do', 'enemy', 'engage', 'for', 'friend', 'function', 'good', 'goodbye', 'guy', 'happy', 'head', 'hear', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hogan', 'hola', 'how', 'hulk', 'i', 'in', 'interesting', 'is', 'jarvis', 'later', 'location', 'looking', 'me', 'miss', 'my', 'name', 'next', 'nice', 'off', 'on', 'pepper', 'perfect', 'pleasehi', 'question', 'quote', 'ready', 'report', 'see', 'some', 'something', 'speaking', 'status', 'sy

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\CSFic\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [32]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-32-1a891a7a8859>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [33]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
15/15 [==============================] - 0s 786us/step - loss: 3.1509 - accuracy: 0.0417
Epoch 2/200
15/15 [==============================] - 0s 857us/step - loss: 3.0413 - accuracy: 0.1806
Epoch 3/200
15/15 [==============================] - 0s 786us/step - loss: 2.9759 - accuracy: 0.1806
Epoch 4/200
15/15 [==============================] - 0s 857us/step - loss: 2.8548 - accuracy: 0.1806
Epoch 5/200
15/15 [==============================] - 0s 714us/step - loss: 2.7412 - accuracy: 0.2083
Epoch 6/200
15/15 [==============================] - 0s 786us/step - loss: 2.6695 - accuracy: 0.2083
Epoch 7/200
15/15 [==============================] - 0s 786us/step - loss: 2.5036 - accuracy: 0.2361
Epoch 8/200
15/15 [==============================] - 0s 714us/step - loss: 2.4282 - accuracy: 0.3056
Epoch 9/200
15/15 [==============================] - 0s 714us/step - loss: 2.3980 - accuracy: 0.3194
Epoch 10/200
15/15 [==============================] - 0s 714us/step - loss: 2.2808 - accura

In [34]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [35]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [36]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
import pyttsx3
from pygame import mixer
import wikipedia
import speech_recognition as sr
import threading
import queue


# function for button to work
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "TS: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "J.A.R.V.I.S.: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        engine = pyttsx3.init()
        engine.say(res)
        engine.runAndWait()

# adding scroll function
def move_window(event):
    root.geometry('+{0}+{1}'.format(event.x_root, event.y_root))    
    
# ability to use enter button   
def onclick(event):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "TS: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "J.A.R.V.I.S.: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        engine = pyttsx3.init()
        engine.say(res)
        engine.runAndWait()
        
# adding hover ability
class ToolTip(object):

    def __init__(self, widget):
        self.widget = widget
        self.tipwindow = None
        self.id = None
        self.x = self.y = 0

    def showtip(self, text):
        "Display text in tooltip window"
        self.text = text
        if self.tipwindow or not self.text:
            return
        x, y, cx, cy = self.widget.bbox("insert")
        x = x + self.widget.winfo_rootx() + 57
        y = y + cy + self.widget.winfo_rooty() +27
        self.tipwindow = tw = Toplevel(self.widget)
        tw.wm_overrideredirect(1)
        tw.wm_geometry("+%d+%d" % (x, y))
        label = Label(tw, text=self.text, justify=LEFT,
                      background="#ffffe0", relief=SOLID, borderwidth=1,
                      font=("tahoma", "8", "normal"))
        label.pack(ipadx=1)

    def hidetip(self):
        tw = self.tipwindow
        self.tipwindow = None
        if tw:
            tw.destroy()

def CreateToolTip(widget, text):
    toolTip = ToolTip(widget)
    def enter(event):
        toolTip.showtip(text)
    def leave(event):
        toolTip.hidetip()
    widget.bind('<Enter>', enter)
    widget.bind('<Leave>', leave)
    
    
# play background song and buttons
def play_music():
    if MusicButton["text"] == "Play":
        MusicButton["text"] = "Stop"
        MusicButton["bg"] = "#aa0505"
        mixer.music.play()
    else:
        MusicButton["text"] = "Play"
        MusicButton["bg"] = "green"
        mixer.music.pause()
        
def change_vol(_=None):
    mixer.music.set_volume(vol.get())
    
    
# speech recognition
def rec():
    r = sr.Recognizer()
    #msg.configure(text="Say something")
    while True:
        with sr.Microphone() as source: 
            r.adjust_for_ambient_noise(source)
            audio = r.listen(source)
        try:
            txt = r.recognize_google(audio)
            EntryBox.configure(text=txt)
            print(txt)
        except Exception as e:
            print(e)
            break

# layout of GUI
base = Tk()
base.title("J.A.R.V.I.S. Chat")
base.configure(bg="black")
base.geometry("800x440")
base.resizable(width=FALSE, height=FALSE)
p1 = PhotoImage(file = 'avg.png')

# Setting icon of master window
base.iconphoto(False, p1)

#Create Chat window
ChatLog = Text(base, bd=0, bg="#fbca03", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Helvetica",10,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#aa0505", activebackground="#6a0c0b",fg='white',
                    command= send)

# add ability to hit enter
base.bind('<Return>', onclick)
SendButton.bind('<Button-1>', onclick)

# add hover text to button
CreateToolTip(SendButton, text = 'Click to Send Text or Push Enter')
SendButton.pack()

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#fbca03", width="29", height="5", font="Arial")

# hover feature for message box
CreateToolTip(EntryBox, text = 'Enter Text Here')
EntryBox.pack()

# add animation
frameCnt = 10
frames = [PhotoImage(file='jarvis.gif',format = 'gif -index %i' %(i)) for i in range(frameCnt)]

def update(ind):
    frame = frames[ind]
    ind += 1
    if ind == frameCnt:
        ind = 0
    label.configure(image=frame)
    base.after(100, update, ind)
    
label = Label(base)
label.pack()
base.after(0, update, 0)

# second animation
frameCnt2 = 26
frames2 = [PhotoImage(file='iron.gif',format = 'gif -index %i' %(i)) for i in range(frameCnt2)]

def update2(ind2):
    frame2 = frames2[ind2]
    ind2 += 1
    if ind2 == frameCnt2:
        ind2 = 0
    label2.configure(image=frame2)
    base.after(135, update2, ind2)
    
label2 = Label(base)
label2.config(bg="black")
label2.pack()
base.after(0, update2, 0)


# add music button
mixer.init()
mixer.music.load("Acdc.mp3")
MusicButton = Button(base, text=' Play Music',font=("Helvetica",10,'bold'), width=14, bg='#aa0505', fg='White', command=play_music)
CreateToolTip(MusicButton, text = 'Click to play')
MusicButton.pack()

# volume control
vol = Scale(
    base,
    from_ = 0,
    to = 1,
    orient = HORIZONTAL ,
    resolution = .1,
    ####################
    command=change_vol
    ####################
)
vol.configure(bg="#aa0505", fg = "white")
CreateToolTip(vol, text = 'Adjust Volume')
vol.pack()

# add labels
lab1 = Label(base, text = "Enter Text Here:")
lab1.config(bg="black", fg = "white", font=("Helvetica",12))
lab1.pack()

# speech button
#EntryBox = Label()
#spchbtn = Button(text="Speech",command=rec)


#Place all components on the screen
scrollbar.place(x=776,y=100, height=285)
ChatLog.place(x=6,y=100, height=285, width=770)
EntryBox.place(x=150, y=395, height=30, width=507)
SendButton.place(x=671, y=395, height=30, width=123)
label.place(x=16, y=6, height=90, width=90)
MusicButton.place(x=690, y=12, height=30, width=90)
vol.place(x=680, y=50)
lab1.place(x=26, y=394)
label2.place(x=330, y=6, height=90, width = 134)
#spchbtn.place(x=26, y=394)


base.mainloop()

TclError: can't use "pyimage297" as iconphoto: not a photo image